In [1]:
from database import Ticket

# Example 1: Create a new ticket
new_ticket_data = {
    "name": "John Doe",
    "email": "john@example.com",
    "subject": "Cannot access dashboard",
    "description": "Detailed description here",
    "language": "en",
    "category": "technical",
    "sentiment": "neutral",
    "strategy": "template",
    "priority": "low"
}

try:
    ticket = Ticket.create_or_update_from_json(new_ticket_data)
    print(f"Created ticket {ticket.id}")
except ValueError as e:
    print(f"Error creating ticket: {e}")

Created ticket 1


In [2]:
# Example 2: Update an existing ticket
update_data = {
    "status": "in_progress",
    "description": "Updated description"
}

try:
    updated_ticket = Ticket.create_or_update_from_json(update_data, ticket_id=1)
    print(f"Updated ticket {updated_ticket.id}")
except ValueError as e:
    print(f"Error updating ticket: {e}")

# Get ticket as dictionary
ticket_dict = ticket.to_dict()

Updated ticket 1


In [2]:
Ticket.get_all_tickets_as_json()

[{'id': 1,
  'name': 'John Doe',
  'email': 'john@example.com',
  'subject': 'Cannot access dashboard',
  'description': 'Detailed description here',
  'created_at': '2024-10-25T22:01:31.838687',
  'updated_at': '2024-10-25T22:01:31.837686',
  'language': 'en',
  'status': 'open',
  'category': 'technical',
  'sentiment': 'neutral',
  'strategy': 'template'}]

In [2]:
import requests
import constants

def query(payload):
	response = requests.post(constants.SENTIMENT_ANALYSIS_MODEL, headers=constants.HF_HEADERS, json=payload)
	return response.json()
	
output = query({
	"inputs": "Intenté hacer un pago con mi tarjeta de crédito, pero recibí un mensaje de error que decía 'Transacción rechazada'. Ya verifiqué que tengo saldo disponible. ¿Pueden ayudarme a solucionarlo?",
})

print(output)
# Sort by score and get the highest scoring label
most_probable = max(output[0], key=lambda x: x['score'])
    
print(most_probable['label'])

[[{'label': 'negative', 'score': 0.7681470513343811}, {'label': 'positive', 'score': 0.13854241371154785}, {'label': 'neutral', 'score': 0.09331050515174866}]]
negative


In [1]:
import models

new_ticket_data = {
    "name": "John Doe",
    "email": "john@example.com",
    "subject": "Cannot access dashboard",
    "description": "Intenté hacer un pago con mi tarjeta de crédito, pero recibí un mensaje de error que decía 'Transacción rechazada'. Ya verifiqué que tengo saldo disponible. ¿Pueden ayudarme a solucionarlo?",
    "language": "en",
    "category": "technical",
    "sentiment": "neutral",
    "strategy": "template",
    "priority": "low"
}

print(models.query_sentiment(new_ticket_data))

('negative', 0.6660830974578857)


In [2]:
import models
print(models.query_language(new_ticket_data))

('spa_Latn', 0.9964954853057861)


In [2]:
prompt = f"""You are an experienced customer support specialist for our platform. Your role is to provide helpful, accurate, and empathetic responses to user tickets.

Context and Guidelines:
- Respond in a professional yet friendly tone
- Address all points raised in the ticket
- Provide specific, actionable solutions
- Include relevant step-by-step instructions when needed
- Express empathy for any issues or frustrations
- Use clear, concise language
- Maintain consistency with our platform's terminology

Current Ticket Information:
{new_ticket_data}

Please craft a response that:
1. Acknowledges the user's concern
2. Provides a clear solution or next steps
3. Includes any relevant troubleshooting steps
4. Offers additional assistance if needed

Response Format:
- Start with a greeting
- Address the main issue
- Provide the solution
- Close with a professional sign-off
- Keep the tone positive and solution-focused

Remember: Focus on providing accurate, helpful information while maintaining a balance between professionalism and approachability.

Generate a response now:"""

models.query_mistral(prompt)

'ncounter issues after following these steps, please contact our dedicated support team at support@example.com or call us at 1-800-123-4567 for further assistance. Our team is available 24/7 to help you with any payment-related queries.\n\nWe understand that this experience may have caused some frustration, and we sincerely apologize for any inconvenience. Our team is committed to ensuring a seamless payment experience for all our users. If you have any questions or concerns, please do not hesitate to reach out to us.\n\nBest regards,\n\n[Your Name]\nCustomer Support Specialist\nExample Inc.'

In [3]:
import constants
prompt = f"""You are an expert support ticket classifier for our platform. Your task is to analyze and categorize the following ticket into exactly one of these categories: {constants.TICKET_CATEGORIES}

Classification Guidelines:
- Analyze the ticket's primary issue, not secondary concerns
- Consider key terms and context specific to each category
- Choose the MOST relevant category if multiple could apply
- Be consistent with categorization patterns

Ticket Content:
{new_ticket_data}

Response Requirements:
- Respond with EXACTLY ONE WORD from the provided categories
- Use lowercase
- No punctuation or additional text
- Must be from the provided category list

Analyze the ticket and respond with just the category name:"""
response = models.query_mistral(prompt)

def get_matching_category(response, valid_categories):
    # Split response into words and check each against valid categories
    words = response.lower().split()
    for word in words:
        if word in valid_categories:
            return word
    return None  # Return None if no matching category found

print(response)

print(get_matching_category(response, constants.TICKET_CATEGORIES))


None
